In [ ]:
!pip install -U transformers langchain chromadb accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.2/834.2 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparin

# Models - Dolly 2.O

In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True, device=0)


A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Both `device` and `device_map` are specified. `device` will override `device_map`. You will most likely encounter unexpected behavior. Please remove `device` and keep `device_map`.


In [ ]:
import inspect

def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

In [ ]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator


class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """ A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function. """

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """ Validate that the input variables are correct. """
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = f"""
        Given the function name and source code, generate an English language explanation of the function in 500 words.
        Function Name: {kwargs["function_name"].__name__}
        Source Code:
        {source_code}
        Explanation:
        """
        return prompt
    
    def _prompt_type(self):
        return "function-explainer"

In [ ]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)


        Given the function name and source code, generate an English language explanation of the function in 500 words.
        Function Name: get_source_code
        Source Code:
        def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

        Explanation:
        


In [ ]:
def fib(n): 
  if n == 0 or n == 1: 
    return n 
  else: 
    return fib(n-1) + fib(n-2)

In [ ]:
def test():


    df = pd.DataFrame({'Name': ['Tom', 'Brad', 'Kyle', 'Jerry'], 
                       'Age': [20, 21, 19, 18], 
                       'Height': [6.1, 5.9, 6.0, 6.1]}) 

    plt.figure(figsize=(14,7)) # set figure size 

    ax1=plt.subplot(2,3,1)#create subplot grid with three rows and four columns 
    sns.barplot(x='Name', y='Height', data=df, ax=ax1 )#make bar plot 

    ax2=plt.subplot(2,3,2)#create subplot grid with three rows and four columns 
    sns.countplot( x='Age',data=df, order=[18,19,20,21])# make count plot 

    plt.show()

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)
prompt = FunctionExplainerPromptTemplate(input_variables=["function_name"])
llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)
print(llm_chain.run(function_name=fib).lstrip())




The Fibonacci sequence is a sequence of numbers generated by performing arithmetic on the previous two numbers in the Fibonacci sequence.  The first ten numbers in the sequence are:
1, 1, 2, 3, 5, 8, 13, 21
The second ten numbers in the sequence are:
2, 2, 3, 5, 8, 13, 21, 34, 55
The next sequence number would be:
3, 3, 5, 8, 13, 21, 34, 55, 89,...

If you sum the two numbers in the Fibonacci sequence, you get the next number in the sequence:
1 + 2 = 3
Fib(3) = 3

Note that when we reach a number N, the function stops computing because the equation N = fib(N) does not have a solution.


However, this function doesn't need to stop at N = 100, since it can start from any number and continue computing fib(100).


In that case, the last line of the function would be:
34 + 55 = 89
That's why we call this function fib(n) where n is a positive integer less than 100.


This function can also be


In [ ]:
print(llm_chain.run(function_name=test).lstrip())

Function Name: test
        Source Code:
        def test():
            df = pd.DataFrame({'Name': ['Tom', 'Brad', 'Kyle', 'Jerry'], 
                                'Age': [20, 21, 19, 18], 
                                'Height': [6.1, 5.9, 6.0, 6.1]}) 
            plt.figure(figsize=(14,7)) # set figure size 
            ax1=plt.subplot(2,3,1)#create subplot grid with three rows and four columns 
            sns.barplot(x='Name', y='Height', data=df, ax=ax1)

            ax2=plt.subplot(2,3,2)#create subplot grid with three rows and four columns 
            sns.countplot( x='Age',data=df, order=[18,19,20,21])

            plt.show()

        Tom
        Brad
        Kyle
        Jerry

        Height


In [ ]:

res = generate_text("Write a function in Javascript to reverse words in a given string.")
print(res[0]["generated_text"])

Write a function in Javascript to reverse words in a given string.
function reverseWords(str) {
  var res = str,
    word,
    index = 0;

  while (index < str.length - 1) {
    word = str.substr(index, 1);
    res = res.split('').reverse().join('');
    index += 1;
  };

  return res;
};


In [ ]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-12b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-12b", device_map="auto", torch_dtype=torch.bfloat16).to('cuda')



In [ ]:
input_text = "How can I write a Python function to generate the nth Fibonacci number?"
inputs = tokenizer.encode(input_text, return_tensors="pt").to('cuda')
outputs = model.generate(inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens= 512)
print(tokenizer.decode(outputs[0]))

How can I write a Python function to generate the nth Fibonacci number?
 fib(n) = fib(n-1) + fib(n-2)

I know that this is the nth Fibonacci number if n is a positive even integer.  How can I generate the nth Fibonacci number for all positive integers n?

A:

You can use the following recursive formula:
fib(n) = (fib(n-1) + fib(n-2)) mod 2

where the modulo operation is done using a 2 as the modulus.

A:

You can use the following formula:
fib(n) = (fib(n-1) + fib(n-2)) % 2

where % is the modulo operator.

<|endoftext|>


In [ ]:
input_text = "what’s wrong with this code: def fib(n): if n == 0 or n == 1: return n"
inputs = tokenizer.encode(input_text, return_tensors="pt").to('cuda')
outputs = model.generate(inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens= 512)
print(tokenizer.decode(outputs[0]))

what’s wrong with this code: def fib(n): if n == 0 or n == 1: return n else: return fib(n-1) + fib(n-2)
 fib(5)
Out[]: 21

I think the problem is that you are not returning the result of the recursive call, you are returning the result of the function call fib(n-1) + fib(n-2) which is the same as fib(n-2) + fib(n-2) which is the same as fib(n-2)
Try this:
def fib(n):
    if n == 0 or n == 1:
        return n
    else:
        return fib(n-1) + fib(n-2)

fib(5)
Out[]: 21

<|endoftext|>
